### [DLM_testing copy.ipynb](vscode-remote://ssh-remote/home/muyang/learned-affordance-constraints/DLM_testing%20copy.ipynb)

Rewrite some of the code from memory, ignoring the output of cells and not exceeding 200 lines.



In [ ]:


{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import copy\n",
    "import collections\n",
    "import functools\n",
    "import os\n",
    "import json\n",
    "import csv\n",
    "\n",
    "import numpy as np\n",
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.nn.functional as F\n",
    "\n",
    "from game.header import *\n",
    "\n",
    "from data.toy_dataset import ToyDataset\n",
    "from torch_geometric.loader import DataLoader"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "def string_to_action_triple(action_string, video_id):\n",
    "    a = action_string.split(' ')\n",
    "    if len(a) == 3:\n",
    "        action_triple = (video_id, int(a[0][1:]), float(a[1]), float(a[2]))\n",
    "    elif len(a) == 1 and a[0] == '':\n",
    "        return None\n",
    "    else:\n",
    "        print('invalid string')\n",
    "        return None\n",
    "    return action_triple\n",
    "\n",
    "root = '/data/Datasets/ag/'\n",
    "actions = []\n",
    "\n",
    "with open(root + 'annotations/Charades/Charades_v1_train.csv') as f:\n",
    "    reader = csv.DictReader(f)\n",
    "    for row in reader:\n",
    "        video_id = row['id']\n",
    "        action_string = row['actions'].split(';')\n",
    "        video_actions = [string_to_action_triple(action, video_id) for action in action_string]\n",
    "        actions.append(video_actions)\n",
    "\n",
    "        print(row['length'])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [],
   "source": [
    "actions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "for action in actions:\n",
    "    pass"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 65,
   "metadata": {},
   "outputs": [],
   "source": [
    "def pyg_to_pred_tensors(data):\n",
    "    def convert_graph(data):\n",
    "        nullary = torch.zeros(len(RELS))\n",
    "        unary = data.x\n",
    "\n",
    "        binary = torch.zeros(data.num_nodes, data.num_nodes, len(RELS))\n",
    "        for i,type in enumerate(data.edge_type):\n",
    "            binary[data.edge_index[0][i], data.edge_index[1][i], type] = 1\n",
    "        \n",
    "        tensors = [nullary, unary, binary]\n",
    "        tensors = [tensor.unsqueeze(0) for tensor in tensors]\n",
    "\n",
    "        return tensors\n",
    "\n",
    "    if data.batch is not None:\n",
    "        datalist = data.to_data_list()\n",
    "        tensors_list = [convert_graph(d) for d in datalist]\n",
    "        nullaries = torch.vstack([t[0] for t in tensors_list])\n",
    "        unaries = torch.vstack([t[1] for t in tensors_list])\n",
    "        binaries = torch.vstack([t[2] for t in tensors_list])\n",
    "        return [nullaries, unaries, binaries]\n",
    "    else:\n",
    "        pred_tensors = convert_graph(data)\n",
    "        return pred_tensors\n",
    "\n",
    "def show_pyg_graph(graph):\n",
    "    import networkx as nx\n",
    "    # Create an empty NetworkX directed graph\n",
    "    G = nx.DiGraph()\n",
    "    \n",
    "    # Add nodes with their features\n",
    "    for i in range(graph.num_nodes):\n",
    "        G.add_node(i, label=NODES[graph.node_type[i].item()])\n",
    "    \n",
    "    # Add edges with their attributes\n",
    "    edge_index = graph.edge_index\n",
    "    edge_type = graph.edge_type if 'edge_type' in graph else None\n",
    "    for i in range(edge_index.size(1)):\n",
    "        source, target = edge_index[:, i].tolist()\n",
    "        if edge_type is not None:\n",
    "            label = RELS[int(edge_type[i].item())]\n",
    "            G.add_edge(source, target, label=label)\n",
    "        else:\n",
    "            G.add_edge(source, target)\n",
    "    \n",
    "    # Draw the graph\n",
    "    pos = nx.circular_layout(G)\n",
    "    labels = nx.get_node_attributes(G, 'label')\n",
    "    edge_labels = nx.get_edge_attributes(G, 'label')\n",
    "    \n",
    "    nx.draw(G, pos, with_labels=True, labels=labels, node_color='lightblue', node_size=500, font_size=10, font_color='black', font_weight='bold', arrows=True)\n",
    "    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}




Made changes.